In [1]:
#Sktime imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sktime.classification.shapelet_based import MrSEQLClassifier
from sklearn.model_selection import GroupShuffleSplit

from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
import sys
from os.path import exists

import importlib

# EEG package modules__________________________________________________________________
import constants
import custom_ts_length
# import eeg_stat_ts
import format_eeg_data
import run_expts

### Load the data -  code from original script used to run expts (run_mv_tsc_expts_mrsql.py)

In [29]:
'''
o 1. PD vs HC,   2.(PD , PD+RBD) vs HC,   3.(PD + PD&RBD) vs HC vs RBD  multiclass, 4.PD vs PD+RBD vs RBD vs HC 
'''
core_path = '/export/sphere/ebirah/ko20929/RBD_files/notebooks/frequency_analysis/constructed_data/band_power_time_series/expt_dfs/'

#1 - This is to run the custom length expts
folders = {'N1': 'N1_customised_40_expt_files/' , 'N2' : 'N2_120_expt_files_v2/','N3' : 'N3_82_expt_files/', 'REM' :'REM_19_expt_files/'}
#2 -This is to run the expts on the full original length data without any dropping or cutting of the TS 
folders = {'N1': 'N1_full_expts/' , 'N2' : 'N2_full_expts/','N3' : 'N3_full_expts/', 'REM' :'REM_full_expts/'}
#3 - This is to run the wakeful eyes closed experiments
folders = {'EC1': 'EC_8_expts/' , 'EC2' : 'EC_full_expts/' }

#Folder = folders[sys.argv[1]] #User can input which experiments will be run
print('Change the folder here ^_^! ')

folder = folders['EC1'] #I'm choosing this because it's the TS & Therefore quickest hopefully?
folder = 'REM_full_expts/'

expts = ['MrSQL_expt_1', 'MrSQL_expt_2' , 'MrSQL_expt_3' , 'MrSQL_expt_4']

power_band_cols = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma', 'TotalAbsPow']

regions = ['Prefrontal','Frontal', 'Left Frontal', 'Right Frontal', 'Central', 'Left Central', 
           'Right Central', 'Left Temporal','Right Temporal', 'Parietal', 'Left Parietal', 'Right Parietal', 'Occipital']

#Load the appropriate X (full set of features ) , y (full set of labels) , groups 
load_path = core_path + folder
X = pd.read_hdf(load_path + 'X.h5', key='df', mode='r')
y = pd.read_hdf(load_path + 'y_full.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + 'groups.h5', key='df', mode='r')  

Change the folder here ^_^! 


In [80]:
subsets_dict_new = {}
for col in power_band_cols:
    name = col + '_' + 'Prefrontal'
    subsets_dict_new[name] = [name]

subsets_dict_new

# subsets_dict_new = {'Theta_Prefrontal': ['Theta_Prefrontal']}

subsets_dict_new = {'Gamma_Prefrontal': ['Gamma_Prefrontal']}

subsets_dict_new

{'Gamma_Prefrontal': ['Gamma_Prefrontal']}

In [81]:
clfs = {'MrSQL_1' : MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) }

In [82]:
results_dict = {}
# for num in [1,2,3,4]:
for num in [1]:
    print('commencing for ' + str(num) + '...')

    #1.Generate expt specific X,y,groups
    X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups,num)

    t1 = time.time()
    results_df = run_expts.run_mv_tsc(X_expt ,y_expt , groups_expt, clfs , return_df = True,subset_names_and_cols = subsets_dict_new, random_states = [13 , 37])
    
    t2 = time.time()
    
    results_dict[num] = results_df
    
    print(t2-t1)

commencing for 1...
19.293851852416992


In [ ]:
mean_acc, acc_std , y_pred_arrays , y_true_arrays = run_expts.run_clf_with_CV(X, y , groups ,  MrSEQLClassifier(seql_mode='fs', symrep=[ 'sfa' , 'sax']) , rand_state = 13)

In [99]:
groups_expt

id_val
0     1101
1     1101
2     1102
3     1103
4     1103
5     1104
6     1104
7     1105
8     1105
9     1106
10    1106
11    1108
12    1108
13    1109
14    1110
15    1111
16    1111
17    1112
18    1113
19    1113
20    1116
21    1116
22    1117
23    1118
24    1119
25    1119
50    4401
51    4401
52    4402
53    4403
54    4403
55    4404
56    4405
57    4406
58    4407
59    4408
60    4408
61    4409
62    4409
63    4410
Name: part_id, dtype: object

In [98]:
y_expt

id_val
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
50    1
51    1
52    1
53    1
54    1
55    1
56    1
57    1
58    1
59    1
60    1
61    1
62    1
63    1
Name: y_label, dtype: int64

In [97]:
X_expt[['Theta_Prefrontal']]

,Theta_Prefrontal
id_val,
0,0 0.276990 1 0.307561 2 0.34982...
1,0 0.219331 1 0.196625 2 0.35440...
2,0 0.264978 1 0.200974 2 0.31222...
3,0 0.208444 1 0.201994 2 0.28776...
4,0 0.240993 1 0.247943 2 0.26621...
5,0 0.203282 1 0.217057 2 0.20701...
6,0 0.173289 1 0.223946 2 0.18677...
7,0 0.176294 1 0.151547 2 0.13519...
8,0 0.173299 1 0.157983 2 0.13791...


In [90]:
subsets_dict_new

{'Gamma_Prefrontal': ['Gamma_Prefrontal']}

In [92]:
list(subsets_dict_new.keys())

['Gamma_Prefrontal']

In [89]:
clfs

{'MrSQL_1': MrSEQLClassifier(symrep=['sfa', 'sax'])}

In [87]:
y_expt

id_val
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
50    1
51    1
52    1
53    1
54    1
55    1
56    1
57    1
58    1
59    1
60    1
61    1
62    1
63    1
Name: y_label, dtype: int64

In [86]:
X_expt

,Delta_Prefrontal,Theta_Prefrontal,Alpha_Prefrontal,Sigma_Prefrontal,Beta_Prefrontal,Gamma_Prefrontal,TotalAbsPow_Prefrontal,Delta_Frontal,Theta_Frontal,Alpha_Frontal,...,Beta_Right Parietal,Gamma_Right Parietal,TotalAbsPow_Right Parietal,Delta_Occipital,Theta_Occipital,Alpha_Occipital,Sigma_Occipital,Beta_Occipital,Gamma_Occipital,TotalAbsPow_Occipital
id_val,,,,,,,,,,,,,,,,,,,,,
0,0 0.451210 1 0.485463 2 0.43161...,0 0.276990 1 0.307561 2 0.34982...,0 0.103973 1 0.097094 2 0.09117...,0 0.067770 1 0.049431 2 0.04941...,0 0.084322 1 0.052745 2 0.06406...,0 0.015734 1 0.007706 2 0.01391...,0 17.695738 1 31.468791 2 24.91...,0 0.311254 1 0.422293 2 0.31824...,0 0.387958 1 0.313285 2 0.40434...,0 0.120272 1 0.122524 2 0.09902...,...,0 0.067770 1 0.044652 2 0.05272...,0 0.021533 1 0.011230 2 0.02127...,0 25.146073 1 35.238018 2 35.89...,0 0.409707 1 0.426703 2 0.40673...,0 0.336839 1 0.364479 2 0.36692...,0 0.115262 1 0.095484 2 0.09464...,0 0.048851 1 0.049100 2 0.04387...,0 0.063414 1 0.049790 2 0.06160...,0 0.025927 1 0.014445 2 0.02621...,0 23.320720 1 29.252003 2 31.82...
1,0 0.556763 1 0.524243 2 0.29737...,0 0.219331 1 0.196625 2 0.35440...,0 0.094027 1 0.110869 2 0.14174...,0 0.039724 1 0.048578 2 0.06565...,0 0.070927 1 0.103004 2 0.12319...,0 0.019228 1 0.016681 2 0.01762...,0 17.473748 1 16.225109 2 21.59...,0 0.450051 1 0.412968 2 0.22212...,0 0.275347 1 0.242244 2 0.41909...,0 0.091611 1 0.092234 2 0.12475...,...,0 0.073545 1 0.105956 2 0.08992...,0 0.030145 1 0.027294 2 0.02515...,0 21.557608 1 19.069051 2 28.72...,0 0.512351 1 0.547930 2 0.29000...,0 0.242246 1 0.187055 2 0.38436...,0 0.102475 1 0.110124 2 0.13093...,0 0.042521 1 0.048592 2 0.06140...,0 0.069114 1 0.083062 2 0.09948...,0 0.031293 1 0.023236 2 0.03381...,0 21.173499 1 20.898948 2 24.27...
2,0 0.518393 1 0.598627 2 0.51664...,0 0.264978 1 0.200974 2 0.31222...,0 0.136418 1 0.110137 2 0.09996...,0 0.036153 1 0.039359 2 0.03237...,0 0.038165 1 0.044136 2 0.03261...,0 0.005893 1 0.006768 2 0.00618...,0 24.203308 1 20.034308 2 22.22...,0 0.597110 1 0.544499 2 0.61713...,0 0.250918 1 0.250465 2 0.24285...,0 0.087326 1 0.113344 2 0.07785...,...,0 0.030180 1 0.028770 2 0.02095...,0 0.003850 1 0.004594 2 0.00324...,0 26.826324 1 24.744472 2 29.54...,0 0.566282 1 0.619565 2 0.62730...,0 0.296454 1 0.223790 2 0.23879...,0 0.084722 1 0.101777 2 0.08596...,0 0.027023 1 0.027680 2 0.02361...,0 0.022117 1 0.023561 2 0.02094...,0 0.003401 1 0.003626 2 0.00338...,0 33.240304 1 32.621830 2 33.75...
3,0 0.579698 1 0.506822 2 0.47951...,0 0.208444 1 0.201994 2 0.28776...,0 0.128702 1 0.196569 2 0.14745...,0 0.042088 1 0.049570 2 0.04641...,0 0.036001 1 0.040105 2 0.03335...,0 0.005067 1 0.004939 2 0.00550...,0 33.520892 1 33.884840 2 29.70...,0 0.585443 1 0.517018 2 0.49464...,0 0.227275 1 0.205606 2 0.26766...,0 0.098601 1 0.170206 2 0.14023...,...,0 0.041781 1 0.036932 2 0.03528...,0 0.005629 1 0.004602 2 0.00506...,0 46.061971 1 58.620364 2 51.32...,0 0.516351 1 0.507897 2 0.44593...,0 0.288186 1 0.266124 2 0.35342...,0 0.124000 1 0.161381 2 0.13571...,0 0.037932 1 0.038381 2 0.03929...,0 0.029148 1 0.022897 2 0.02206...,0 0.004382 1 0.003319 2 0.00356...,0 57.580677 1 76.092224 2 65.10...
4,0 0.637609 1 0.585811 2 0.55187...,0 0.240993 1 0.247943 2 0.26621...,0 0.075810 1 0.111487 2 0.11748...,0 0.021807 1 0.028935 2 0.02894...,0 0.020956 1 0.022830 2 0.03044...,0 0.002825 1 0.002994 2 0.00503...,0 50.313122 1 50.539509 2 32.16...,0 0.612979 1 0.519274 2 0.46867...,0 0.242006 1 0.270989 2 0.31765...,0 0.074152 1 0.126578 2 0.12447...,...,0 0.025540 1 0.026344 2 0.03680...,0 0.003158 1 0.002725 2 0.00517...,0 74.258674 1 77.538462 2 47.39...,0 0.611242 1 0.578125 2 0.44989...,0 0.214953 1 0.236279 2 0.33584...,0 0.121409 1 0.133276 2 0.15061...,0 0.025846 1 0.027853 2 0.03471...,0 0.023621 1 0.021931 2 0.02431...,0 0.002928 1 0.002535 2 0.00461...,0 73.568298 1 77.591422 2 56...
5,0 0.620811 1 0.561290 2 0.54175...,0 0.203282 1 0.217057 2 0.20701...,0 0.059101 1 0.08

In [84]:
acc_std_results = {}
# for num in [1,2,3,4]:
for num in [1]:
    run_expts.generate_subset_acc_std(results_df, return_df = True)
    acc_std_results_df = run_expts.generate_subset_acc_std(results_dict[num], return_df = True)
    acc_std_results[num] = acc_std_results_df
acc_std_results[1]

,random_state,Gamma_Prefrontal_MrSQL_1_acc,Gamma_Prefrontal_MrSQL_1_std
0,13,0.930000,0.09798
1,37,0.910556,0.04542


In [64]:
acc_std_results[1]

,random_state,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std
0,13,0.925000,0.150000
1,37,0.935556,0.052775


,random_state,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std
0,13,0.925000,0.150000
1,37,0.935556,0.052775


,random_state,Delta_Prefrontal_MrSQL_1_acc,Delta_Prefrontal_MrSQL_1_std
0,13,0.950000,0.100000
1,37,0.955556,0.054433


,random_state,Alpha_Prefrontal_MrSQL_1_acc,Alpha_Prefrontal_MrSQL_1_std
0,13,0.950000,0.100000
1,37,0.955556,0.054433


,random_state,Gamma_Prefrontal_MrSQL_1_acc,Gamma_Prefrontal_MrSQL_1_std
0,13,0.930000,0.09798
1,37,0.910556,0.04542


In [58]:
# all_results = acc_std_results[1].copy() This df contains the full results
all_results

,random_state,Delta_Prefrontal_MrSQL_1_acc,Delta_Prefrontal_MrSQL_1_std,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std,Alpha_Prefrontal_MrSQL_1_acc,Alpha_Prefrontal_MrSQL_1_std,Sigma_Prefrontal_MrSQL_1_acc,Sigma_Prefrontal_MrSQL_1_std,Beta_Prefrontal_MrSQL_1_acc,Beta_Prefrontal_MrSQL_1_std,Gamma_Prefrontal_MrSQL_1_acc,Gamma_Prefrontal_MrSQL_1_std,TotalAbsPow_Prefrontal_MrSQL_1_acc,TotalAbsPow_Prefrontal_MrSQL_1_std
0,13,0.950000,0.100000,0.552857,0.167703,0.567302,0.113256,0.56373,0.090866,0.525079,0.133376,0.507857,0.177430,0.538175,0.177977
1,37,0.955556,0.054433,0.510556,0.147242,0.659444,0.178882,0.61500,0.139660,0.490556,0.166441,0.579444,0.226075,0.577222,0.175052


In [70]:
#all results mark 2 

,random_state,Delta_Prefrontal_MrSQL_1_acc,Delta_Prefrontal_MrSQL_1_std,Theta_Prefrontal_MrSQL_1_acc,Theta_Prefrontal_MrSQL_1_std,Alpha_Prefrontal_MrSQL_1_acc,Alpha_Prefrontal_MrSQL_1_std,Sigma_Prefrontal_MrSQL_1_acc,Sigma_Prefrontal_MrSQL_1_std,Beta_Prefrontal_MrSQL_1_acc,Beta_Prefrontal_MrSQL_1_std,Gamma_Prefrontal_MrSQL_1_acc,Gamma_Prefrontal_MrSQL_1_std,TotalAbsPow_Prefrontal_MrSQL_1_acc,TotalAbsPow_Prefrontal_MrSQL_1_std
0,13,0.950000,0.100000,0.552857,0.167703,0.567302,0.113256,0.56373,0.090866,0.525079,0.133376,0.507857,0.177430,0.538175,0.177977
1,37,0.955556,0.054433,0.510556,0.147242,0.659444,0.178882,0.61500,0.139660,0.490556,0.166441,0.579444,0.226075,0.577222,0.175052
